# Making a job script for HIV data. 

In [ ]:
# Output folder for scripts
output_dir = "../jobs/"
mkdir(output_dir)

In [9]:
# Define the sample IDs
sample_ids = ["703010505", "703010848"]

# Start the script content
script_content = """
#!/bin/bash -l
#SBATCH --job-name="MPL"
hostname date
mkdir -p /HIV

date
hostname
# ----------- Start Main Computation ---------------#
g++ src/main.cpp src/inf.cpp src/io.cpp -O3 -march=native -lgslcblas -lgsl -o bin/mpl
"""

# Add a line for each sample ID
for id in sample_ids
    script_content *= """
./bin/mpl -d HIV -i $(id)-3-poly-seq2state.dat -o $(id)-3-poly-seq2state-MPL.dat -g 1e5 -N 1e4 -m Zanini-extended.dat -sc covariance-$(id)-3-poly-seq2state.dat -sn numerator-$(id)-3-poly-seq2state.dat
./bin/mpl -d HIV -i $(id)-3-poly-seq2state.dat -o $(id)-3-poly-seq2state-SL.dat  -g 1e5 -N 1e4 -m Zanini-extended.dat -nc
"""
end

# Add footer
script_content *= """
date
hostname
exit 0
"""

# Write to file
script_path = output_dir * "run_mpl_HIV.sh"
open( Write, "w") do file
    write(file, script_content)
end

# Make the file executable
run(`chmod +x $script_path`)


Process(`chmod +x ../jobs/run_mpl_HIV.sh`, ProcessExited(0))

# Making a job script for SHIV data. 

In [10]:
# Dictionary: group name => list of sample IDs
datasets = Dict(
    "CH505" => ["703010505", "RM5695", "RM6070", "RM6072", "RM6697", "RM6699", "RM6701", "RM6703"],
    "CH848" => ["RM6163", "RM6167", "RM6700", "RM6713", "RM6714", "RM6720", "703010848"]
)

# Loop over both groups
for (group, ids) in datasets
    # Initialize the script content
    script_content = """
#!/bin/bash -l
#SBATCH --job-name="MPL_$(group)"
hostname date
mkdir -p /HIV/$(group)

date
hostname
# ----------- Start Main Computation ---------------#
"""

    # Add commands for each ID in the group
    for id in ids
        script_content *= "./bin/mpl -d HIV/$(group) -i $(id)-poly.num -o $(id)-poly-AA-MPL.dat -g 1e5 -N 1e4 -q 22 -m mutation_rate.txt -sc covariance-$(id)-poly-AA.dat -sn numerator-$(id)-poly-AA.dat\n"
        script_content *= "./bin/mpl -d HIV/$(group) -i $(id)-poly.num -o $(id)-poly-AA-SL.dat  -g 1e5 -N 1e4 -q 22 -m mutation_rate.txt -nc\n"
    end

    # Add footer
    script_content *= """
date
hostname
exit 0
"""

    # Define script file name
    script_path = output_dir * "run_mpl_SHIV_$(group).sh"

    # Write the script to file
    open(script_path, "w") do file
        write(file, script_content)
    end

    # Make it executable
    run(`chmod +x $script_path`)
end
